In [ ]:
import requests
import json
from datetime import datetime
import pandas as pd
import os

def buscar_cnpj(cnpj):
    url = f"https://www.receitaws.com.br/v1/cnpj/{cnpj}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def salvar_informacoes(dados, nome_arquivo):
    with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
        arquivo.write(f"Informações da empresa (CNPJ: {dados['cnpj']})\n")
        arquivo.write(f"Data da consulta: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n\n")
        
        campos = [
            "nome", "fantasia", "logradouro", "numero", "complemento",
            "municipio", "uf", "cep", "telefone", "email",
            "situacao", "data_situacao", "cnae_principal",
            "capital_social", "porte", "natureza_juridica"
        ]
        
        for campo in campos:
            if campo in dados:
                arquivo.write(f"{campo.replace('_', ' ').title()}: {dados[campo]}\n")
        
        arquivo.write("\nAtividades secundárias:\n")
        for atividade in dados.get('atividades_secundarias', []):
            arquivo.write(f"- {atividade['text']}\n")

def processar_arquivo_excel(caminho_arquivo):
    try:
        df = pd.read_excel(caminho_arquivo, usecols=[0])  # Lê apenas a primeira coluna
        df.columns = ['CNPJ']  # Renomeia a coluna para 'CNPJ'
        
        resultados = []
        total = len(df)
        
        for index, row in df.iterrows():
            cnpj = str(row['CNPJ']).zfill(14)
            print(f"Processando CNPJ {index+1}/{total}: {cnpj}")
            
            dados = buscar_cnpj(cnpj)
            if dados:
                nome_arquivo = f"info_empresa_{cnpj}.txt"
                salvar_informacoes(dados, nome_arquivo)
                print(f"Informações salvas no arquivo: {nome_arquivo}")
                resultados.append({'CNPJ': cnpj, 'Status': 'Sucesso'})
            else:
                print(f"Não foi possível obter informações para o CNPJ: {cnpj}")
                resultados.append({'CNPJ': cnpj, 'Status': 'Falha'})
        
        # Cria um DataFrame com os resultados e salva em um novo arquivo Excel
        df_resultados = pd.DataFrame(resultados)
        nome_arquivo_resultados = "resultados_processamento.xlsx"
        df_resultados.to_excel(nome_arquivo_resultados, index=False)
        print(f"Resumo do processamento salvo em: {nome_arquivo_resultados}")
        
    except Exception as e:
        print(f"Erro ao processar o arquivo: {str(e)}")

def main():
    while True:
        escolha = input("Escolha uma opção:\n1 - Processar CNPJ único\n2 - Processar arquivo Excel\n3 - Sair\nSua escolha: ")
        
        if escolha == '1':
            cnpj = input("Digite o CNPJ (apenas números): ")
            cnpj = ''.join(filter(str.isdigit, cnpj))
            
            if len(cnpj) != 14:
                print("CNPJ inválido. Por favor, digite 14 dígitos.")
                continue
            
            print("Buscando informações...")
            dados = buscar_cnpj(cnpj)
            
            if dados:
                nome_arquivo = f"info_empresa_{cnpj}.txt"
                salvar_informacoes(dados, nome_arquivo)
                print(f"Informações salvas no arquivo: {nome_arquivo}")
            else:
                print("Não foi possível obter as informações. Verifique o CNPJ e tente novamente.")
        
        elif escolha == '2':
            caminho_arquivo = input("Digite o caminho completo do arquivo Excel: ")
            if os.path.exists(caminho_arquivo):
                processar_arquivo_excel(caminho_arquivo)
            else:
                print("Arquivo não encontrado. Verifique o caminho e tente novamente.")
        
        elif escolha == '3':
            print("Encerrando o programa...")
            break
        
        else:
            print("Opção inválida. Por favor, escolha 1, 2 ou 3.")

if __name__ == "__main__":
    main()